# Klassifikation von Bildern mit Deep Learning

_Philipp Rapp_

FOM Mechatronik Vorlesung 2022

In [ ]:
# Unsere Standard Pakete
import numpy as np
import matplotlib.pyplot as plt

# Fortschrittsbalken
from tqdm import tqdm

# Tensorflow
import tensorflow as tf

# Funktionen und Klassen fuer die Layers (Schichten) des Deep Neural Networks
import tensorflow.nn as nn
from tensorflow.keras.layers import Dense, MaxPool2D, Conv2D, Flatten

# Datensatz
from tensorflow.keras.datasets import cifar10

# Loss und Optimierer
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import SGD

%matplotlib inline

## Laden des Datensatzes

Wir laden die Daten von der (ehemaligen) Universitäts-Homepage von Alexander Krishevsky
unter https://www.cs.toronto.edu/~kriz/cifar.html

Konkret wird die Datei https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz heruntergeladen.

Die Daten werden lokal gespeichert unter `/Users/<username>/.keras/datasets` (macOS) bzw. `/home/<username>/.keras/datasets` (Linux).


Der CIFAR-10 Datensatz besteht aus 60000 Bildern, welche wiederum jeweils einer von 10 Klassen zugeordnet sind.
50000 Bilder sind Trainingsdaten, 10000 Bilder sind Testdaten.
Jedes Bild hat ein Label. Dies wird benötigt, da wir _supervised learning_ betreiben.

Wir bezeichnen die eigentlichen Bilddaten (die den Eingang in unser neuronales Netz bilden) mit $x$.
Die Labels bezeichnen wir mit $y$.
Damit kann man die Funktion des Netzes mathematisch beschreiben als $y=f(x)$.

Weitere Details befinden sich im Tech Report "Learning Multiple Layers of Features from Tiny Images, Alex Krizhevsky, 2009.".

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
# Definition der Klassen
#classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
classes = ['Flugzeug', 'Auto', 'Vogel', 'Katze', 'Wild', 'Hund', 'Frosch', 'Pferd', 'Schiff', 'LKW']



In [ ]:
# Wir verifizieren dass wir tatsaechlich 50000 Trainings Samples und 10000 Test Samples erhalten haben.
# Jedes Sample ist ein 32x32 Pixel Bild mit RGB Werten.
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
# Konvertierung von uint8 auf float
img = x_train[0]
print(img[:,:,0])
print(img[:,:,0].shape)
print(img.dtype)

x_train = x_train / 255.0
x_test = x_test / 255.0

img = x_train[0]
print(img[:,:,0])
print(img[:,:,0].shape)
print(img.dtype)

In [ ]:
# Visualisierung
def imshow(img):
    plt.imshow(img)

fig = plt.figure(figsize=(25,20))
for idx in np.arange(20):
    ax = fig.add_subplot(4, 20//4, idx+1, xticks=[], yticks=[])
    imshow(x_train[idx,:,:,:])
    ax.set_title(classes[y_train[idx].item()])

## Data Loaders

Die sog. data loaders helfen uns dabei, die Daten (Input und Label) häppchenweise zu laden.
Dabei lassen sich die Daten auch noch randomisiert durchmischen, damit wir _Stochastic Gradient Descent_ implementieren können.

API Dokumentation: https://www.tensorflow.org/guide/data#basic_mechanics

### Training data loader

Hier wollen wir das Durchmischen haben.

In [ ]:
dataset_train_x = tf.data.Dataset.from_tensor_slices(x_train)
print(f'Length of the training data set samples is {len(dataset_train_x)}')

dataset_train_y = tf.data.Dataset.from_tensor_slices(y_train.squeeze())
print(f'Length of the training data set labels is {len(dataset_train_y)}')

# Jetzt verknupfen wir die Samples und die Labels.
# Dies ermoeglicht es uns, ueber die Trainingsdaten zu iterieren und jedes Mal
# Eingangsdaten (Samples) mit passenden Labels zu erhalten.
# Wichtig ist hierbei, dass dieser Schritt vor der Durchmischung (Shuffling)
# passieren muss.
dataset_train = tf.data.Dataset.zip((dataset_train_x, dataset_train_y))

# Jetzt setzen wir die Groesse der Samples, die pro Schritt verarbeitet werden.
# Dies ist die (Mini) Batch Size.
batch_size = 20
dataset_train = dataset_train.batch(batch_size)
print(f'Length of the training data set is {len(dataset_train)} for a batch size of {batch_size}')

In [ ]:
# Durchmischen des Datensatzes
dataset_train = dataset_train.shuffle(batch_size)

In [ ]:
# Test des Data Loaders.
# Wir erwarten bei jedem Aufruf unterschiedliche Ergebnisse,
# wobei Sample (Input) und Label immer zueinander passen.

dataset_train_iter = iter(dataset_train)
images, labels = next(dataset_train_iter)

fig = plt.figure(figsize=(25,4))
for idx, (image, label) in enumerate(zip(images, labels)):
    ax = fig.add_subplot(2, 20//2, idx+1, xticks=[], yticks=[])
    imshow(image)
    ax.set_title(classes[label])


### Data loader fuer die Testdaten

Hier brauchen wir keine Durchmischung.

In [ ]:
dataset_test_x = tf.data.Dataset.from_tensor_slices(x_test)
dataset_test_y = tf.data.Dataset.from_tensor_slices(y_test.squeeze())

dataset_test = tf.data.Dataset.zip((dataset_test_x, dataset_test_y))
dataset_test = dataset_test.batch(batch_size)

## Definition der Architektur des Convolutional Neural Networks (CNN)

API Dokumentation:
* Allgemein
* Conv2d function:
* Conv2D layer:

Architektur des CNNs:
* Conv layer

**Hinweis**: Die Operation in den Conv Layerns ist eine _Convolution_ (Faltung).
Das ist die mathematische Operation, die wir in der DRT fuer die Beschreibung des Uebertragungsverhaltens eines LTI-Systems im Zeitbereich (d.h. insbesondere nicht im Frequenzbereich) kennen gelernt haben.

Das LTI-System entspricht hier dem Filterkern, der die Bildverarbeitungsoperation durchfuehrt (z.B. Kantendetektion oder auch Extraktion abstrakterer Merkmale). Die Faltung ist zweidimensional, da es sich um Bilddaten handelt.
Der Zeitbereich entspricht hier dem Pixelbereich.

Wichtig ist, dass der Filterkern nicht haendisch vorgegeben wird (z.B. als Kantenfilter), sondern durch Loesung eines Optimierungsproblems berechnet (_gelernt_) wird.

In [ ]:
class Net(tf.Module):
    
    def __init__(self, name=None):
        self.conv1 = Conv2D(filters=16, kernel_size=3, padding='same')
        self.conv2 = Conv2D(filters=32, kernel_size=3, padding='same')
        self.conv3 = Conv2D(filters=64, kernel_size=3, padding='same')
        self.flatten = Flatten()
        self.fc1 = Dense(512)
        self.fc2 = Dense(10)
        
    # Vorwaertspfad
    def __call__(self, x):
        # Erster Conv layer
        x = nn.relu(self.conv1(x))
        x = nn.max_pool2d(x, ksize=(2, 2), strides=(2, 2), padding='VALID')
        
        # Zweiter Conv layer
        x = nn.relu(self.conv2(x))
        x = nn.max_pool2d(x, ksize=(2, 2), strides=(2, 2), padding='VALID')
        
        
        # Dritter Conv layer
        x = nn.relu(self.conv3(x))
        x = nn.max_pool2d(x, ksize=(2, 2), strides=(2, 2), padding='VALID')
        
        # Flatten layer.
        # Die Features, die immer noch (wie das Bild) zweidimensional vorliegen
        # (wenn auch mit niedrigerer Aufloesung), werden nun einen (eindimensionalen)
        # Vektor konvertiert.
        x = self.flatten(x)
        
        # Zwei dichte (fully-connected) layer.
        x = nn.relu(self.fc1(x))
        x = nn.log_softmax(self.fc2(x))
        
        return x

model = Net()
log_ps = model(images)

# Sicherstellen, dass die summe des softmax 1.0 ist,
# da es sich um Wahrscheinlichkeiten handelt.
# ps ... probabilities
ps = tf.exp(log_ps)
print(np.sum(ps, axis=1))

## Testen des _untrainierten_ Netzwerkes

Wir erwarten, dass das untrainierte Netz in ca. 10% der Faelle ein Bild richtig klassifiziert.
Dies ist die Erfolgsquote bei Raten, da es 10 Klassen gibt.

In [ ]:
# Korrekt klassifizierte Bilder.
correctly_classified = 0
total_samples = 0

for images, labels in dataset_test:
    log_ps = model(images)
    # Praedizierte Klasse
    pred = np.argmax(log_ps, axis=1)
    correctly_classified += np.sum(pred == labels.numpy())
    total_samples += len(labels)

print(f'Gesamtanzahl an Samples: {total_samples}')
print(f'Korrekt klassifizierte Samples: {correctly_classified}')
print(f'Accuracy: {correctly_classified / total_samples * 100} Prozent')

## Definition des Optimierers und der Kostenfunktion (Loss)



In [ ]:
criterion = CategoricalCrossentropy()
optimizer = SGD(lr=0.003)

def loss(model, x_input, y_labels):
    log_ps = model(x_input)
    ps = tf.exp(log_ps)
    return criterion(y_true=tf.one_hot(y_labels, len(classes)), y_pred=ps)

def train_step(model, x_input, y_labels):
    with tf.GradientTape() as tape:
        loss_value = loss(model, x_input, y_labels)
    # Berechnung des Gradienten (das ist die Jacobi-Matrix des Losses
    # bzgl. der Parameter oder Gewichte des Netzes).
    gradients = tape.gradient(loss_value, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss_value

# print(loss(model, images, labels))
# train_step(model, images, labels)

## Trainieren des Netzes

In [ ]:
n_epochs = 2

print(f'Trainiere fuer {n_epochs} Epochen')
for epoch in range(1, n_epochs+1):
    print(f'Epoche {epoch}')
    train_loss = 0.0
    
    for data, target in tqdm(dataset_train):
        train_loss += train_step(model, data, target)
        
    train_loss /= len(dataset_train) * batch_size
    
    print(f'Training loss: {train_loss}')

## Test des _trainierten_ Netzes

In [ ]:
# Korrekt klassifizierte Bilder.
correctly_classified = 0
total_samples = 0

for images, labels in dataset_test:
    log_ps = model(images)
    # Praedizierte Klasse
    pred = np.argmax(log_ps, axis=1)
    correctly_classified += np.sum(pred == labels.numpy())
    total_samples += len(labels)

print(f'Gesamtanzahl an Samples: {total_samples}')
print(f'Korrekt klassifizierte Samples: {correctly_classified}')
print(f'Accuracy: {correctly_classified / total_samples * 100} Prozent')

In [ ]:
log_ps = model(images)
pred = np.argmax(log_ps, axis=1)

fig = plt.figure(figsize=(25,20))
for idx, (image, label, predicted) in enumerate(zip(images, labels, pred)):
    ax = fig.add_subplot(4, 20//4, idx+1, xticks=[], yticks=[])
    imshow(image)
    col = "green" if predicted == label else "red"
    ax.set_title(f'{classes[predicted]} ({classes[label]})', color=col, fontsize=20)